In [1]:
#这里的代码是使用lightgbm 对预测dt的特征做特征重要性排序。

import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.cross_validation import train_test_split


pd.options.mode.chained_assignment = None  # default='warn'


train_list = ['180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
          '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
          '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
          '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
          '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015']

valid_list = ['181016', '181017', '181018', '181019', '181020', '181021']

# test_list = ['180709', '180710', '180711', '180712', '180713', '180714', '180715']

# d_list= ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
#           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
#           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
#           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
#           '180711', '180712', '180713', '180714', '180715',\
#          '180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725', \
#           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804', \
#           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
#           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822',\
#           '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
#           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
#           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
#           '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
#           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
#           '181016', '181017', '181018', '181019', '181020', '181021']


day_all = ['180501', '180502', '180503', '180504', '180505', '180506', '180507', '180508', '180509', '180510',\
           '180511', '180512', '180513', '180514', '180515', '180516', '180517', '180518', '180519', '180520',\
           '180521', '180522', '180523', '180524', '180525', '180526', '180527', '180528', '180529', '180530',\
           '180531','180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
           '180711', '180712', '180713', '180714', '180715',\
           '180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725',\
           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', \
            '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
            '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021','181022']

c:\program files\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#定义截取函数
def over_data_do(s):
    if s<0:
        return 0
    elif s> 2880:
        return 2880
    else:
        return s

In [3]:

 #选取第几轮数据切片
for n_i in range(1,20):
    train_df = []
    print(n_i)
    for day in train_list:
        print(day,end=' ')   
        day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train\train_%s_edt.csv'%day)

        #day_all_data = day_all_data[day_all_data.end_time.notnull()]

        #过滤结束日期超过2天的
        day_after_2 = day_all[day_all.index(day)+1]
        day_after_2_int = int('20'+day_after_2+'235959')
        day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
        day_all_data = day_all_data[day_all_data.end_time.notnull()]

        round_len = day_all_data.shape[0]//20
        train_df.append(day_all_data[n_i*round_len:(n_i+1)*round_len])

    train = pd.concat(train_df, ignore_index=True)

    train.to_csv('./data_edt_train/edt_split_%d.csv'%n_i,index=False)

1
180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 2
180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 3
180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 

In [4]:
# 和下面的代码功能同

n_i = 3
train = pd.read_csv('./data_edt_train/edt_split_%d.csv'%n_i)

valid_df = []
for day in valid_list:
    print(day,end=' ')
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train\train_%s_edt.csv'%day)
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    day_after_2 = day_all[day_all.index(day)+1]
    day_after_2_int = int('20'+day_after_2+'235959')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    
    valid_df.append(day_all_data.sample(frac=0.2))
    
valid = pd.concat(valid_df, ignore_index=True)

print(train.shape)
print(valid.shape)

181016 181017 181018 181019 181020 181021 (357701, 215)
(166703, 215)


In [4]:
#读取数据 按天

# 划分训练集， 验证集 测试集

train_df = []
n_i = 1  #选取第几轮数据切片
for day in train_list:
    print(day,end=' ')   
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train\train_%s_edt.csv'%day)
    
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    
    #过滤结束日期超过2天的
    day_after_2 = day_all[day_all.index(day)+1]
    day_after_2_int = int('20'+day_after_2+'235959')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    day_all_data = day_all_data[day_all_data.end_time.notnull()]
    
    round_len = day_all_data.shape[0]//14
    train_df.append(day_all_data[n_i*round_len:(n_i+1)*round_len])
    
train = pd.concat(train_df, ignore_index=True)

train.to_csv('./data_edt_train/edt_split_%d.csv'%n_i,index=False)

valid_df = []
for day in valid_list:
    print(day,end=' ')
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train\train_%s_edt.csv'%day)
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    day_after_2 = day_all[day_all.index(day)+1]
    day_after_2_int = int('20'+day_after_2+'235959')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    
    valid_df.append(day_all_data.sample(frac=0.2))
    
valid = pd.concat(valid_df, ignore_index=True)

# test_df = []

# for day in test_list:
#     print(day,end=' ')   
#     day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train/train_%s_edt.csv'%day)
#     day_all_data = day_all_data[day_all_data.end_time.notnull()]
#     test_df.append(day_all_data)
    
# test = pd.concat(test_df, ignore_index=True)

#空值处理

180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 181016 181017 181018 181019 181020 181021 

IndexError: list index out of range

In [29]:

#过滤训练不需要的数据字段
cols_train =[clm for clm in valid.columns if clm not in ['order_id','job_name','v_date','order_d','from_time_1000_start',\
                                                            'start_time','end_time','order_id','job_name','day_0_bef_start_time',\
                                                            'day_0_bef_end_time','day_bef_0_dt']]


valid_x = valid[cols_train]
valid_y = valid['day_0_bef_end_time']

train_x,test_x,train_y,test_y = train_test_split(train[cols_train],train['day_0_bef_end_time'],random_state=33,test_size=0.1)

train_y = train_y.fillna(train_y.mean())
valid_y = valid_y.fillna(valid_y.mean())
test_y = test_y.fillna(test_y.mean())  

print(train_x.shape)
print(test_x.shape)
print(valid_x.shape)

(321930, 206)
(35771, 206)
(166703, 206)


In [8]:
# lightgbm 算法预测
import json
import lightgbm as lgb



y_train = train_y.values
y_test = test_y.values

X_train = train_x.values
X_test = test_x.values

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,silent=True)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


# specify your configurations as a dict
params = {'task': 'train',
   'boosting_type': 'gbdt',
   'objective': 'regression',
   'metric':  'mse',
   'num_leaves': 2**7,
   'learning_rate': 0.1,
   'feature_fraction': 0.9,
   'bagging_fraction': 0.8,
   'bagging_freq': 5,
   'verbose': 50,
    'verbose_eval':50}
  
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

print('Save model...')

# save model to file
from sklearn.externals import joblib
joblib.dump(gbm,'edt_gbm.pkl')
clf = joblib.load("edt_gbm.pkl")

gbm.save_model('lightgbm_model_edt.txt')
print('Start predicting...')


# predict

y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
vlid_pred = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
#clf_pred = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
# eval print(y_pred)
print('The MAE of prediction train_data is:', mean_absolute_error(y_test, y_pred))
print('The MAE of prediction valid_data is:', mean_absolute_error(valid_y, vlid_pred))
# print('The mean_absolute_error of prediction valid_data is:', mean_absolute_error(valid_y, clf_pred))

print('The MSE of prediction train_data is:', mean_squared_error(valid_y, vlid_pred))


Start training...
[1]	valid_0's l2: 130509
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 114246
[3]	valid_0's l2: 100763
[4]	valid_0's l2: 89778.6
[5]	valid_0's l2: 80626.8
[6]	valid_0's l2: 73181.3
[7]	valid_0's l2: 67069.6
[8]	valid_0's l2: 61919.6
[9]	valid_0's l2: 57680.2
[10]	valid_0's l2: 54145.3
[11]	valid_0's l2: 51102.5
[12]	valid_0's l2: 48577.5
[13]	valid_0's l2: 46468.9
[14]	valid_0's l2: 44630.7
[15]	valid_0's l2: 43069.6
[16]	valid_0's l2: 41700.1
[17]	valid_0's l2: 40535.3
[18]	valid_0's l2: 39510.2
[19]	valid_0's l2: 38658.1
[20]	valid_0's l2: 37857.9
[21]	valid_0's l2: 37131.5
[22]	valid_0's l2: 36521.1
[23]	valid_0's l2: 35892
[24]	valid_0's l2: 35390.5
[25]	valid_0's l2: 34890.7
[26]	valid_0's l2: 34449.7
[27]	valid_0's l2: 34080.7
[28]	valid_0's l2: 33721.3
[29]	valid_0's l2: 33382.4
[30]	valid_0's l2: 33076.5
[31]	valid_0's l2: 32834.7
[32]	valid_0's l2: 32544.9
[33]	valid_0's l2: 32337.5
[34]	valid_0's l2: 32174.5
[35]	valid_0's l

In [ ]:
The MAE of prediction train_data is: 91.7078200717
The MAE of prediction valid_data is: 86.6426869845
The MSE of prediction train_data is: 23181.5505251

In [9]:


### 特征选择   保存模型特征重要性
df = pd.DataFrame(train_x.columns.tolist(), columns=['feature'])
df['importance']=list(gbm.feature_importance())                           # 特征分数
df = df.sort_values(by='importance',ascending=False)                      # 特征排序
df.to_csv("./edt_feature_score.csv",index=None,encoding='gbk')  # 保存分数

In [27]:
features = pd.read_csv('edt_feature_score.csv')
features = features['feature'][:70]

In [ ]:
# lightgbm 算法预测
import json
import lightgbm as lgb



y_train = train_y.values
y_test = test_y.values

X_train = train_x.values
X_test = test_x.values

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,silent=True)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


# specify your configurations as a dict
params = {'task': 'train',
   'boosting_type': 'gbdt',
   'objective': 'regression',
   'metric':  'mse',
   'num_leaves': 2**7,
   'learning_rate': 0.1,
   'feature_fraction': 0.9,
   'bagging_fraction': 0.8,
   'bagging_freq': 5,
   'verbose': 50,
    'verbose_eval':50}
  
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

print('Save model...')

# save model to file
from sklearn.externals import joblib
joblib.dump(gbm,'edt_gbm.pkl')
clf = joblib.load("edt_gbm.pkl")

gbm.save_model('lightgbm_model_edt.txt')
print('Start predicting...')


# predict

y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
vlid_pred = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
#clf_pred = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
# eval print(y_pred)
print('The MAE of prediction train_data is:', mean_absolute_error(y_test, y_pred))
print('The MAE of prediction valid_data is:', mean_absolute_error(valid_y, vlid_pred))
# print('The mean_absolute_error of prediction valid_data is:', mean_absolute_error(valid_y, clf_pred))

print('The MSE of prediction valid_data is:', mean_squared_error(valid_y, vlid_pred))

In [30]:

import xgboost as xgb

dtr = xgb.XGBRegressor(max_depth=8, learning_rate=0.1)


# train_x,test_x,train_y,test_y
dtr.fit(train_x[features],train_y)

y_pred = dtr.predict(test_x[features])
vlid_pred = dtr.predict(valid_x[features])

print('The MAE of prediction train_data is:', mean_absolute_error(test_y, y_pred))
print('The MAE of prediction valid_data is:', mean_absolute_error(valid_y, vlid_pred))
#print('The mean_absolute_error of prediction valid_data is:', mean_absolute_error(valid_y, clf_pred))

print('The MSE of prediction valid_data is:', mean_squared_error(valid_y, vlid_pred))

ValueError: Found input variables with inconsistent numbers of samples: [71541, 35771]

In [31]:
print('The MAE of prediction train_data is:', mean_absolute_error(test_y, y_pred))
print('The MAE of prediction valid_data is:', mean_absolute_error(valid_y, vlid_pred))
#print('The mean_absolute_error of prediction valid_data is:', mean_absolute_error(valid_y, clf_pred))

print('The MSE of prediction valid_data is:', mean_squared_error(valid_y, vlid_pred))

The MAE of prediction train_data is: 85.7944480783
The MAE of prediction valid_data is: 85.1375072346
The MSE of prediction valid_data is: 23907.0302866


In [16]:
dtr = xgb.XGBRegressor(max_depth=8, learning_rate=0.1)
所有特征
The MAE of prediction train_data is: 85.0395749341
The MAE of prediction valid_data is: 85.0483190648
The MSE of prediction valid_data is: 23211.1046788
max_depth=9
The MAE of prediction train_data is: 79.2312313776
The MAE of prediction valid_data is: 84.0364825932
The MSE of prediction valid_data is: 22906.597829
max_depth=10
The MAE of prediction train_data is: 74.7287435372
The MAE of prediction valid_data is: 85.8226865694
The MSE of prediction valid_data is: 23641.0085073

The MAE of prediction train_data is: 74.7287435372
The MAE of prediction valid_data is: 85.8226865694
The MSE of prediction valid_data is: 23641.0085073


In [ ]:
dtr = xgb.XGBRegressor(max_depth=8, learning_rate=0.1)

100
The MAE of prediction train_data is: 84.8834261761
The MAE of prediction valid_data is: 87.1222425463
The MSE of prediction valid_data is: 23468.1188846
120
The MAE of prediction train_data is: 84.9453742738
The MAE of prediction valid_data is: 90.8959669376
The MSE of prediction valid_data is: 24453.9947591
80
The MAE of prediction train_data is: 85.1988766365
The MAE of prediction valid_data is: 85.5486905383
The MSE of prediction valid_data is: 23312.3482064
70    
The MAE of prediction train_data is: 85.4171723916
The MAE of prediction valid_data is: 85.4214605148
The MSE of prediction valid_data is: 23688.5333861